In [2]:
!pip show torch
!pip show transformers

Name: torch
Version: 1.11.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /home/elice/anaconda3/envs/nightsky/lib/python3.9/site-packages
Requires: typing-extensions
Required-by: kobert
Name: transformers
Version: 4.19.4
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache
Location: /home/elice/anaconda3/envs/nightsky/lib/python3.9/site-packages
Requires: pyyaml, numpy, requests, tokenizers, regex, filelock, huggingface-hub, packaging, tqdm
Required-by: kobert


In [3]:
!pip show gluonnlp tqdm
!pip show mxnet
!pip show sentencepiece

Name: gluonnlp
Version: 0.10.0
Summary: MXNet Gluon NLP Toolkit
Home-page: https://github.com/dmlc/gluon-nlp
Author: Gluon NLP Toolkit Contributors
Author-email: mxnet-gluon@amazon.com
License: Apache-2.0
Location: /home/elice/anaconda3/envs/nightsky/lib/python3.9/site-packages
Requires: numpy, packaging, cython
Required-by: kobert
---
Name: tqdm
Version: 4.64.0
Summary: Fast, Extensible Progress Meter
Home-page: https://tqdm.github.io
Author: 
Author-email: 
License: MPLv2.0, MIT Licences
Location: /home/elice/anaconda3/envs/nightsky/lib/python3.9/site-packages
Requires: 
Required-by: transformers, huggingface-hub
Name: mxnet
Version: 1.9.1
Summary: Apache MXNet is an ultra-scalable deep learning framework. This version uses openblas and MKLDNN.
Home-page: https://github.com/apache/incubator-mxnet
Author: 
Author-email: 
License: Apache 2.0
Location: /home/elice/anaconda3/envs/nightsky/lib/python3.9/site-packages
Requires: graphviz, numpy, requests
Required-by: kobert
Name: sentencepi

In [5]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-v_wau67h/kobert-tokenizer_5c10df18a0fc41459674dcc206648060
  Running command git clone -q https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-v_wau67h/kobert-tokenizer_5c10df18a0fc41459674dcc206648060
  Resolved https://github.com/SKTBrain/KoBERT.git to commit e1f2f37055e7460d8427f6912579c0162cb69831
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-9kctt50p
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-9kctt50p
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit e1f2f37055e7460d8427f6912579c0162cb69831


In [6]:
from kobert.pytorch_kobert import get_pytorch_kobert_model
from kobert_tokenizer import KoBERTTokenizer

tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel, vocab = get_pytorch_kobert_model()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


using cached model. /home/elice/nightsky/.cache/kobert_v1.zip
using cached model. /home/elice/nightsky/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
import pandas as pd

torch.cuda.is_available()
torch.cuda.get_device_name(0)

'Tesla T4'

In [30]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from transformers import BertModel

device = torch.device("cuda:0")

In [9]:
import pandas as pd

data = pd.read_csv("~/nightsky/data/sentiment_data.csv")
data.head()

,Unnamed: 0,Sentence,Emotion,From
0,0,언니 동생으로 부르는게 맞는 일인가요..??,공포,단발성 대화
1,1,그냥 내 느낌일뿐겠지?,공포,단발성 대화
2,2,아직너무초기라서 그런거죠?,공포,단발성 대화
3,3,유치원버스 사고 낫다던데,공포,단발성 대화
4,4,근데 원래이런거맞나요,공포,단발성 대화


In [10]:
data.loc[data['Emotion'] == "공포", "Emotion"] = 0
data.loc[data['Emotion'] == "놀람", "Emotion"] = 1
data.loc[data['Emotion'] == "분노", "Emotion"] = 2
data.loc[data['Emotion'] == "슬픔", "Emotion"] = 3
data.loc[data['Emotion'] == "중립", "Emotion"] = 4
data.loc[data['Emotion'] == "행복", "Emotion"] = 5
data.loc[data['Emotion'] == "혐오", "Emotion"] = 6

In [11]:
data_list = []
for ques, label in zip(data['Sentence'], data['Emotion']):
    data = []
    data.append(ques)
    data.append(str(label))
    data_list.append(data)

In [12]:
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad, pair=pair)
    
    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]
  
  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))
  
  def __len__(self):
    return (len(self.labels))

In [13]:
# setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [14]:
# train 데이터, test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, shuffle=True, random_state=34)

In [15]:
train = []
test = []
for sent, emo in dataset_train:
  train.append(emo)

for sent, emo in dataset_test:
  test.append(emo)

In [16]:
print(train.count('0'), train.count('1'), train.count('2'), train.count('3'), train.count('4'), train.count('5'), train.count('6'))
print(test.count('0'), test.count('1'), test.count('2'), test.count('3'), test.count('4'), test.count('5'), test.count('6'))

5353 8961 9644 7449 38640 8530 6089
1356 2263 2453 1881 9629 2096 1489


In [17]:
tok = tokenizer.tokenize
data_train = BERTDataset(dataset_train, 0, 1, tok, vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, vocab, max_len, True, False)

In [18]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

In [19]:
class BERTClassifier(nn.Module):
  def __init__(self,
               bert,
               hidden_size = 768,
               num_classes = 7,  # 클래스 수 조정
               dr_rate = None,
               params = None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate

    self.classifier = nn.Linear(hidden_size, num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)

  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()
  
  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)
    _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
    if self.dr_rate:
      out = self.dropout(pooler)
    return self.classifier(out)

In [20]:
# BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to('cuda')

# optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/home/elice/anaconda3/envs/nightsky/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
# 정확도 측정을 위한 함수 정의
def calc_accuracy(X, Y):
  max_vals, max_indices = torch.max(X, 1)
  train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
  return train_acc


In [23]:
train_history=[]
test_history=[]
loss_history=[]
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to('cuda')
        segment_ids = segment_ids.long().to('cuda')
        valid_length= valid_length
        label = label.long().to('cuda')
        out = model(token_ids, valid_length, segment_ids)
         
        #print(label.shape,out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    #train_history.append(train_acc / (batch_id+1))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to('cuda')
        segment_ids = segment_ids.long().to('cuda')
        valid_length= valid_length
        label = label.long().to('cuda')
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

/tmp/ipykernel_38526/3425429383.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/1323 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.011458158493042 train acc 0.15625
epoch 1 batch id 201 loss 1.4347476959228516 train acc 0.39715485074626866
epoch 1 batch id 401 loss 1.1932883262634277 train acc 0.46146352867830426
epoch 1 batch id 601 loss 1.1753244400024414 train acc 0.5087354409317804
epoch 1 batch id 801 loss 0.9008404016494751 train acc 0.540398720349563
epoch 1 batch id 1001 loss 0.9893471002578735 train acc 0.5632336413586414
epoch 1 batch id 1201 loss 0.9809057116508484 train acc 0.5789836594504579
epoch 1 train acc 0.587963370213986


/tmp/ipykernel_38526/3425429383.py:31: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


  0%|          | 0/331 [00:00<?, ?it/s]

...

epoch 4 test acc 0.7052803802146944


  0%|          | 0/1323 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.47967395186424255 train acc 0.875
epoch 5 batch id 201 loss 0.4694196581840515 train acc 0.8511349502487562
epoch 5 batch id 401 loss 0.36079779267311096 train acc 0.852906795511222
epoch 5 batch id 601 loss 0.4792831540107727 train acc 0.8573991264559068
epoch 5 batch id 801 loss 0.31139782071113586 train acc 0.8626131398252185
epoch 5 batch id 1001 loss 0.32045096158981323 train acc 0.8664616633366633
epoch 5 batch id 1201 loss 0.42124640941619873 train acc 0.8679095545378851
epoch 5 train acc 0.8677163154533845


  0%|          | 0/331 [00:00<?, ?it/s]

epoch 5 test acc 0.7047440460885774


In [24]:
def predict(predict_sentence):
  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to('cuda')
    segment_ids = segment_ids.long().to('cuda')
    valid_length = valid_length
    label = label.long().to('cuda')
    out = model(token_ids, valid_length, segment_ids)

    test_eval = []

    for i in out:
      logits = i
      logits = logits.detach().cpu().numpy()

      if np.argmax(logits) == 0:
        test_eval.append("공포가")
      elif np.argmax(logits) == 1:
        test_eval.append("놀람이")
      elif np.argmax(logits) == 2:
          test_eval.append("분노가")
      elif np.argmax(logits) == 3:
          test_eval.append("슬픔이")
      elif np.argmax(logits) == 4:
          test_eval.append("중립이")
      elif np.argmax(logits) == 5:
          test_eval.append("행복이")
      elif np.argmax(logits) == 6:
          test_eval.append("혐오가")

  print(">> 입력하신 내용에서 " + test_eval[0] + " 느껴집니다.")

In [25]:
end = 1
while end == 1:
  sentence = input("입력: ")
  if sentence == "0":
    break
  predict(sentence)
  print("\n")

입력: 짜증나


/home/elice/anaconda3/envs/nightsky/lib/python3.9/site-packages/torch/utils/data/dataloader.py:487: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


>> 입력하신 내용에서 분노가 느껴집니다.


입력: 오늘은 gpu가 사용되지 않는 에러가 나서 그거 고치는데 시간이 너무 오래 걸렸다. vm은 진짜 해도해도 어려운 것 같다. 딴 것도 좀 했어야 됐는데...에러 해결하는데 너무 오래 걸려서 아쉽다.
>> 입력하신 내용에서 슬픔이 느껴집니다.


입력: 0


In [28]:
def prediction(predict_sentence):
  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to('cuda')
    segment_ids = segment_ids.long().to('cuda')
    valid_length = valid_length
    label = label.long().to('cuda')
    out = model(token_ids, valid_length, segment_ids)

    test_eval = []

    for i in out:
      logits = i
      logits = logits.detach().cpu().numpy()

      print("공포: {0}, 놀람: {1}, 분노: {2}, 슬픔: {3}, 중립: {4}, 행복: {5}, 혐오: {6}"
        .format(logits[0], logits[1], logits[2], logits[3], logits[4], logits[5], logits[6]))

In [41]:
diary = input()
prediction(diary)

아 운동 좀 해야 되는데 맨날 해야지 말만 하고 안하네..
공포: -0.762182354927063, 놀람: -0.7514278888702393, 분노: -1.6687642335891724, 슬픔: 5.567875862121582, 중립: 0.6801250576972961, 행복: -1.4966859817504883, 혐오: -1.5363456010818481


In [33]:
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

bert.embeddings.position_ids 	 torch.Size([1, 512])
bert.embeddings.word_embeddings.weight 	 torch.Size([8002, 768])
bert.embeddings.position_embeddings.weight 	 torch.Size([512, 768])
bert.embeddings.token_type_embeddings.weight 	 torch.Size([2, 768])
bert.embeddings.LayerNorm.weight 	 torch.Size([768])
bert.embeddings.LayerNorm.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.self.query.weight 	 torch.Size([768, 768])
bert.encoder.layer.0.attention.self.query.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.self.key.weight 	 torch.Size([768, 768])
bert.encoder.layer.0.attention.self.key.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.self.value.weight 	 torch.Size([768, 768])
bert.encoder.layer.0.attention.self.value.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.output.dense.weight 	 torch.Size([768, 768])
bert.encoder.layer.0.attention.output.dense.bias 	 torch.Size([768])
bert.encoder.layer.0.attention.output.LayerNorm.weight 	 torch.Size([768])
bert

In [35]:
PATH = "state_dict_model.pt"

torch.save(model.state_dict(), PATH)

In [37]:
PATH = "entire_model.pt"
torch.save(model, PATH)

In [38]:
model1 = torch.load(PATH)
model1.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [42]:
def predict_test(predict_sentence):
  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, vocab, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)

  model1.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long().to('cuda')
    segment_ids = segment_ids.long().to('cuda')
    valid_length = valid_length
    label = label.long().to('cuda')
    out = model1(token_ids, valid_length, segment_ids)

    test_eval = []

    for i in out:
      logits = i
      logits = logits.detach().cpu().numpy()

      print("공포: {0}, 놀람: {1}, 분노: {2}, 슬픔: {3}, 중립: {4}, 행복: {5}, 혐오: {6}"
        .format(logits[0], logits[1], logits[2], logits[3], logits[4], logits[5], logits[6]))

In [43]:
diary = input()
predict_test(diary)

프리벳 가 4번지에 사는 더즐리 부부는 자신들이 정상적이라는 것을 아주 자랑스럽게 여기는 사람들이었다. 그들은 기이하거나 신비스러운 일과는 전혀 무관해 보였다. 아니, 그런 터무니없는 것은 도저히 참아 내지 못했다.
공포: -2.9716684818267822, 놀람: 1.6539909839630127, 분노: -1.473585844039917, 슬픔: 0.42691534757614136, 중립: 3.574535608291626, 행복: 1.092940092086792, 혐오: -2.1773369312286377
